In [6]:
import torch
from typing import Callable
import time

In [17]:
x = torch.normal(mean=0, std=1, size=(10000, 10000)) * 1e-5
x[0][0] = 1e-23

x = x.abs()
k = 6
mean = torch.mean(x)
std = torch.std(x)
min, max = torch.aminmax(x)
print('max: {}, upper bound: {}'.format(max, mean + k * std))
print('mean: {}, std: {}'.format(mean, std))
if max > mean + k * std:
    print('[FAULT DETECTED]')
else:
    print('No fault detected')

max: 5.4050193284638226e-05, upper bound: 4.4150088797323406e-05
mean: 7.978936992003582e-06, std: 6.028524694556836e-06
[FAULT DETECTED]


In [24]:
x = torch.normal(mean=0, std=1, size=(10000, 10000)) * 1e-5
x[0][0] = 1e-23

x = x.abs()
group_size = 100
x = x.view(-1, group_size, group_size).amax(dim=(1,2))
x = x.where(x != 0, x.max()).log10()
k = torch.tensor(6)

mean = torch.mean(x)
std = torch.std(x)
min, max = torch.aminmax(x)
print('min: {}, max: {}'.format(min, max))
print('lowerBound: {}, upperBound: {}'.format(mean - torch.log10(k) * std, mean + torch.log10(k) * std))
if max > mean + torch.log10(k) * std or min < mean - torch.log10(k) * std:
    print('[FAULT DETECTED]')
else:
    print('No fault detected')

min: -4.4820733070373535, max: -4.2472100257873535
lowerBound: -4.42048978805542, upperBound: -4.373188495635986
[FAULT DETECTED]
